## 美国职业棒球数据分析U报告 1985-2015

In [32]:
START_YEAR = 1985

读取球队数据

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
  
origin_df = pd.read_csv('./baseballdatabank-master/core/Teams.csv')
print origin_df.columns
years_df = origin_df.set_index('yearID')
teams_df = origin_df.set_index('teamID')

Index([u'yearID', u'lgID', u'teamID', u'franchID', u'divID', u'Rank', u'G',
       u'Ghome', u'W', u'L', u'DivWin', u'WCWin', u'LgWin', u'WSWin', u'R',
       u'AB', u'H', u'2B', u'3B', u'HR', u'BB', u'SO', u'SB', u'CS', u'HBP',
       u'SF', u'RA', u'ER', u'ERA', u'CG', u'SHO', u'SV', u'IPouts', u'HA',
       u'HRA', u'BBA', u'SOA', u'E', u'DP', u'FP', u'name', u'park',
       u'attendance', u'BPF', u'PPF', u'teamIDBR', u'teamIDlahman45',
       u'teamIDretro'],
      dtype='object')


读取薪资数据

In [34]:
salaries_df = pd.read_csv('./baseballdatabank-master/core/Salaries.csv')
salaries_df = salaries_df[salaries_df['yearID']>=START_YEAR]
print salaries_df


       yearID teamID lgID   playerID    salary
0        1985    ATL   NL  barkele01    870000
1        1985    ATL   NL  bedrost01    550000
2        1985    ATL   NL  benedbr01    545000
3        1985    ATL   NL   campri01    633333
4        1985    ATL   NL  ceronri01    625000
5        1985    ATL   NL  chambch01    800000
6        1985    ATL   NL  dedmoje01    150000
7        1985    ATL   NL  forstte01    483333
8        1985    ATL   NL  garbege01    772000
9        1985    ATL   NL  harpete01    250000
10       1985    ATL   NL  hornebo01   1500000
11       1985    ATL   NL  hubbagl01    455000
12       1985    ATL   NL  mahleri01    407500
13       1985    ATL   NL  mcmurcr01    275000
14       1985    ATL   NL  mumphje01    775000
15       1985    ATL   NL  murphda05   1625000
16       1985    ATL   NL  oberkke01    616667
17       1985    ATL   NL  perezpa01    450000
18       1985    ATL   NL  perryge01    120000
19       1985    ATL   NL  ramirra01    750000
20       1985

#### 计算球队数据的相关性

In [35]:
grp_salaries = salaries_df[['yearID', 'teamID', 'salary']].groupby(['yearID', 'teamID'])

In [36]:
print grp_salaries.describe()

              salary                                                    \
               count          mean           std       min         25%   
yearID teamID                                                            
1985   ATL      22.0  6.730455e+05  3.970351e+05  120000.0   451250.00   
       BAL      22.0  5.254869e+05  3.189283e+05   60000.0   390625.00   
       BOS      25.0  4.359024e+05  2.985073e+05  115000.0   177500.00   
       CAL      28.0  5.152819e+05  3.052799e+05   60000.0   321666.75   
       CHA      21.0  4.688656e+05  3.440961e+05   60000.0   195000.00   
       CHN      22.0  5.774053e+05  3.190828e+05  200000.0   335000.25   
       CIN      22.0  3.799962e+05  3.227896e+05   60000.0   126875.00   
       CLE      20.0  3.275833e+05  2.290844e+05  110000.0   197500.00   
       DET      20.0  5.174072e+05  2.863597e+05   76000.0   250000.00   
       HOU      20.0  4.996525e+05  3.119774e+05   60000.0   294166.75   
       KCA      22.0  4.236900e+05  3.

In [37]:
#computed_grp_salaries_df = grp_salaries['salary'].agg([np.sum, np.std, np.mean, np.min, np.max]).reset_index()
computed_grp_salaries_df = grp_salaries['salary'].agg([pd.Series.count, np.sum, np.std, np.mean, np.max]).reset_index()
print computed_grp_salaries_df

     yearID teamID  count        sum           std          mean      amax
0      1985    ATL     22   14807000  3.970351e+05  6.730455e+05   1625000
1      1985    BAL     22   11560712  3.189283e+05  5.254869e+05   1472819
2      1985    BOS     25   10897560  2.985073e+05  4.359024e+05   1075000
3      1985    CAL     28   14427894  3.052799e+05  5.152819e+05   1100000
4      1985    CHA     21    9846178  3.440961e+05  4.688656e+05   1242333
5      1985    CHN     22   12702917  3.190828e+05  5.774053e+05   1450000
6      1985    CIN     22    8359917  3.227896e+05  3.799962e+05   1071429
7      1985    CLE     20    6551666  2.290844e+05  3.275833e+05   1100000
8      1985    DET     20   10348143  2.863597e+05  5.174072e+05   1083333
9      1985    HOU     20    9993051  3.119774e+05  4.996525e+05   1350000
10     1985    KCA     22    9321179  3.009853e+05  4.236900e+05   1000000
11     1985    LAN     23   10967917  3.275862e+05  4.768660e+05   1270000
12     1985    MIN     19

In [38]:
team_with_salaries_df = pd.merge(computed_grp_salaries_df, origin_df[['yearID', 'teamID', 'Rank', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'Ghome', 'attendance']], how='inner', on=['yearID', 'teamID']).set_index(['yearID', 'teamID'])

调整球队排名的计算。把季后赛成绩和常规赛成绩统一到一个排名字段ARank，后面以此为依据计算和排名相关的变量

In [39]:
def rank_adjust(row):
    rank = row['Rank'] + 3
    if row['WSWin'] == 'Y':
        rank -= 3
    elif row['LgWin'] == 'Y':
        rank -= 2
    return rank

team_with_salaries_df['ARank'] = team_with_salaries_df.apply(lambda row: rank_adjust(row), axis=1)
print team_with_salaries_df

               count        sum           std          mean      amax  Rank  \
yearID teamID                                                                 
1985   ATL        22   14807000  3.970351e+05  6.730455e+05   1625000     5   
       BAL        22   11560712  3.189283e+05  5.254869e+05   1472819     4   
       BOS        25   10897560  2.985073e+05  4.359024e+05   1075000     5   
       CAL        28   14427894  3.052799e+05  5.152819e+05   1100000     2   
       CHA        21    9846178  3.440961e+05  4.688656e+05   1242333     3   
       CHN        22   12702917  3.190828e+05  5.774053e+05   1450000     4   
       CIN        22    8359917  3.227896e+05  3.799962e+05   1071429     2   
       CLE        20    6551666  2.290844e+05  3.275833e+05   1100000     7   
       DET        20   10348143  2.863597e+05  5.174072e+05   1083333     3   
       HOU        20    9993051  3.119774e+05  4.996525e+05   1350000     3   
       KCA        22    9321179  3.009853e+05  4.236

In [40]:
print team_with_salaries_df.loc[2014]

        count        sum           std          mean      amax  Rank DivWin  \
teamID                                                                        
ARI        26   97861500  3.823383e+06  3.763904e+06  11000000     5      N   
ATL        24   97609000  4.779448e+06  4.067042e+06  14250000     2      N   
BAL        28  103416000  4.349065e+06  3.693429e+06  15000000     1      Y   
BOS        31  139019929  4.887357e+06  4.484514e+06  16000000     5      N   
CHA        24   81830500  4.150391e+06  3.409604e+06  15000000     4      N   
CHN        27   65522500  2.446508e+06  2.426759e+06  11000000     5      N   
CIN        28  108217500  3.914747e+06  3.864911e+06  12000000     4      N   
CLE        26   82151899  4.192258e+06  3.159688e+06  15000000     3      N   
COL        30   95403500  4.090896e+06  3.180117e+06  16000000     4      N   
DET        23  152855500  7.276494e+06  6.645891e+06  22000000     1      Y   
HOU        20   35116300  2.888056e+06  1.755815e+06

In [41]:
print 'arank corr team salaries:', team_with_salaries_df['ARank'].corr(team_with_salaries_df['sum'])

arank corr team salaries: -0.24897004885052806


In [42]:
print 'arank corr team Ghome', team_with_salaries_df['ARank'].corr(team_with_salaries_df['Ghome'])

arank corr team Ghome 0.018350805778188012


In [43]:
print 'arank corr team home attendances', team_with_salaries_df['ARank'].corr(team_with_salaries_df['attendance'])

arank corr team home attendances -0.4336584805405877


In [44]:
print 'salaries corr team home attendances', team_with_salaries_df['sum'].corr(team_with_salaries_df['attendance'])

salaries corr team home attendances 0.5712660270644402


In [45]:
print 'arank corr team players:', team_with_salaries_df['ARank'].corr(team_with_salaries_df['count'])

arank corr team players: 0.015906237476913093


In [46]:
def rolling_years_corr(rolling_size, team_with_salaries_df):
    team_with_salaries_df_sal = team_with_salaries_df['sum']
    team_with_salaries_df[str(rolling_size)+'ysal_avg'] = team_with_salaries_df_sal.rolling(rolling_size).mean()
    team_with_salaries_df_arank = team_with_salaries_df['ARank']
    team_with_salaries_df[str(rolling_size)+'yarank_avg'] = team_with_salaries_df_arank.rolling(rolling_size).mean()

In [47]:
rolling_years_corr(3, team_with_salaries_df)
rolling_years_corr(5, team_with_salaries_df)
print team_with_salaries_df.loc[2014]

        count        sum           std          mean      amax  Rank DivWin  \
teamID                                                                        
ARI        26   97861500  3.823383e+06  3.763904e+06  11000000     5      N   
ATL        24   97609000  4.779448e+06  4.067042e+06  14250000     2      N   
BAL        28  103416000  4.349065e+06  3.693429e+06  15000000     1      Y   
BOS        31  139019929  4.887357e+06  4.484514e+06  16000000     5      N   
CHA        24   81830500  4.150391e+06  3.409604e+06  15000000     4      N   
CHN        27   65522500  2.446508e+06  2.426759e+06  11000000     5      N   
CIN        28  108217500  3.914747e+06  3.864911e+06  12000000     4      N   
CLE        26   82151899  4.192258e+06  3.159688e+06  15000000     3      N   
COL        30   95403500  4.090896e+06  3.180117e+06  16000000     4      N   
DET        23  152855500  7.276494e+06  6.645891e+06  22000000     1      Y   
HOU        20   35116300  2.888056e+06  1.755815e+06

In [48]:
print 'arank corr team Ghome rolling 5 years', team_with_salaries_df['5yarank_avg'].corr(team_with_salaries_df['5ysal_avg'])

arank corr team Ghome rolling 5 years -0.3215991285050088


In [49]:
print 'arank corr team Ghome rolling 3 years', team_with_salaries_df['3yarank_avg'].corr(team_with_salaries_df['3ysal_avg'])

arank corr team Ghome rolling 3 years -0.28920052289936754


#### 计算球员数据的相关性

In [137]:
def add_player_birthyear(laries_df, player_df):
    return salaries_df.join(player_df[['birthYear']], on='playerID', how='inner', rsuffix='_p')
    
def calc_player_age_to_salaries_table(salaries_df):
    return salaries_df.apply(lambda row: row['yearID']-row['birthYear'], axis=1)

salaries_df = add_player_birthyear(salaries_df, pd.read_csv('./baseballdatabank-master/core/Master.csv').set_index('playerID'))
salaries_df['age'] = calc_player_age_to_salaries_table(salaries_df)



Empty DataFrame
Columns: [yearID, teamID, lgID, playerID, salary, birthYear, age]
Index: []
       yearID teamID lgID   playerID   salary  birthYear   age
0        1985    ATL   NL  barkele01   870000     1955.0  30.0
552      1986    ATL   NL  barkele01   880000     1955.0  31.0
1291     1987    ATL   NL  barkele01   890000     1955.0  32.0
1613     1987    ML4   AL  barkele01    72500     1955.0  32.0
1917     1988    ATL   NL  barkele01   900000     1955.0  33.0
1        1985    ATL   NL  bedrost01   550000     1957.0  28.0
1071     1986    PHI   NL  bedrost01   650000     1957.0  29.0
1750     1987    PHI   NL  bedrost01  1050000     1957.0  30.0
2374     1988    PHI   NL  bedrost01   925000     1957.0  31.0
3069     1989    PHI   NL  bedrost01  1450000     1957.0  32.0
4026     1990    SFN   NL  bedrost01  1450000     1957.0  33.0
4485     1991    MIN   AL  bedrost01  1350000     1957.0  34.0
5611     1993    ATL   NL  bedrost01   500000     1957.0  36.0
6534     1994    ATL   NL 

In [138]:
salaries_df[['birthYear', 'age']] = salaries_df[['birthYear', 'age']].astype('int')
print salaries_df

       yearID teamID lgID   playerID   salary  birthYear  age
0        1985    ATL   NL  barkele01   870000       1955   30
552      1986    ATL   NL  barkele01   880000       1955   31
1291     1987    ATL   NL  barkele01   890000       1955   32
1613     1987    ML4   AL  barkele01    72500       1955   32
1917     1988    ATL   NL  barkele01   900000       1955   33
1        1985    ATL   NL  bedrost01   550000       1957   28
1071     1986    PHI   NL  bedrost01   650000       1957   29
1750     1987    PHI   NL  bedrost01  1050000       1957   30
2374     1988    PHI   NL  bedrost01   925000       1957   31
3069     1989    PHI   NL  bedrost01  1450000       1957   32
4026     1990    SFN   NL  bedrost01  1450000       1957   33
4485     1991    MIN   AL  bedrost01  1350000       1957   34
5611     1993    ATL   NL  bedrost01   500000       1957   36
6534     1994    ATL   NL  bedrost01   750000       1957   37
7418     1995    ATL   NL  bedrost01   750000       1957   38
2       

计算场均BATTING数据
G              Games
AB             At Bats
R              Runs
H              Hits
2B             Doubles
3B             Triples
HR             Homeruns
RBI            Runs Batted In
SB             Stolen Bases
CS             Caught Stealing
BB             Base on Balls
SO             Strikeouts
IBB            Intentional walks
HBP            Hit by pitch
SH             Sacrifice hits
SF             Sacrifice flies
GIDP           Grounded into double plays


In [ ]:
batting_df = pd.read_csv('./baseballdatabank-master/core/Batting.csv')
batting_df = batting_df[batting_df['yearID']>=START_YEAR]
print batting_df

In [31]:

def gameavg_batting(row):
    games = row['G']
    if games == 0:
        games = 1
    gv_ab = row['AB']/games
    gv_h = row['H']/games
    gv_2b = row['2B']/games
    gv_3b = row['3B']/games
    gv_hr = row['HR']/games
    gv_rbi = row['RBI']/games
    gv_sb = row['SB']/games
    gv_cs = row['CS']/games
    gv_bb = row['3B']/games
    gv_so = row['SO']/games
    gv_ibb = row['IBB']/games
    gv_hbp = row['HBP']/games
    gv_sh = row['SH']/games
    gv_sf = row['SF']/games
    gv_gidp = row['SO']/games
    return pd.Series({'yearID':row['yearID'], 'playerID':row['playerID'], 'teamID':row['teamID'], 'gv_ab':gv_ab, 
            'gv_h':gv_h, 
            'gv_2b':gv_2b,
            'gv_3b':gv_3b,
            'gv_hr':gv_hr,
            'gv_rbi':gv_rbi,
            'gv_sb':gv_sb,
            'gv_cs':gv_cs,
            'gv_bb':gv_bb,
            'gv_so':gv_so,
            'gv_ibb':gv_ibb,
            'gv_hbp':gv_hbp,
            'gv_sh':gv_sh,
            'gv_sf':gv_sf,
            'gv_gidp':gv_gidp})
print batting_df.apply(lambda row: gameavg_batting(row), axis=1)   
#batting_df[['gv_ab','gv_h','gv_2b','gv_3b','gv_hr','gv_rbi','gv_sb','gv_cs','gv_bb','gv_so','gv_ibb','gv_hbp','gv_sh','gv_sf','gv_gidp']] = batting_df.apply(lambda row: gameavg_batting(row), axis=1)
#print batting_df

           gv_2b     gv_3b     gv_ab     gv_bb     gv_cs   gv_gidp      gv_h  \
0       0.000000  0.000000  4.000000  0.000000  0.000000  0.000000  0.000000   
1       0.240000  0.000000  4.720000  0.000000  0.040000  0.000000  1.280000   
2       0.137931  0.172414  4.724138  0.172414  0.034483  0.172414  1.379310   
3       0.370370  0.074074  4.925926  0.074074  0.037037  0.074074  1.629630   
4       0.440000  0.120000  4.800000  0.120000  0.080000  0.040000  1.560000   
5       0.166667  0.083333  4.083333  0.083333  0.083333  0.083333  0.916667   
6       0.000000  0.000000  4.000000  0.000000  0.000000  0.000000  1.000000   
7       0.322581  0.290323  5.064516  0.290323  0.193548  0.032258  2.032258   
8       1.000000  0.000000  5.000000  0.000000  0.000000  0.000000  1.000000   
9       0.111111  0.055556  4.777778  0.055556  0.000000  0.000000  0.722222   
10      0.045455  0.454545  4.045455  0.454545  0.045455  0.181818  1.227273   
11      0.000000  0.000000  3.000000  0.

In [21]:
'gv_ab, gv_h, gv_2b, gv_3b, gv_hr, gv_rbi, gv_sb, gv_cs, gv_bb, gv_so, gv_ibb, gv_hbp, gv_sh, gv_sf, gv_gidp'.split(', ')

['gv_ab',
 'gv_h',
 'gv_2b',
 'gv_3b',
 'gv_hr',
 'gv_rbi',
 'gv_sb',
 'gv_cs',
 'gv_bb',
 'gv_so',
 'gv_ibb',
 'gv_hbp',
 'gv_sh',
 'gv_sf',
 'gv_gidp']